In [ ]:
# for mooring, can we just interpolate the gridded fields directly rather than interpolating to points and then re-gridding?
# YES but it's slower (!?!)
X = ds.XC.isel(time=0) 
Y = ds.YC.isel(time=0)
i = ds.i
j = ds.j
z = ds.Z.isel(time=0)
k = ds.k
f_x = interpolate.interp1d(X[0,:].values, i)
f_y = interpolate.interp1d(Y[:,0].values, j)
f_z = interpolate.interp1d(z, k, bounds_error=False)

#
model_xav = ds.XC.isel(time=0, j=0).mean(dim='i').values
model_yav = ds.YC.isel(time=0, i=0).mean(dim='j').values
xmooring = model_xav # default lat/lon is the center of the domain
ymooring = model_yav
zmooring_TS = [-1, -10, -50, -100] # depth of T/S instruments
zmooring_UV = [-1, -10, -50, -100] # depth of U/V instruments

#
# ds.Theta.sel(k=1)
# ds.Theta.sel(k=1)
ki = f_z(zmooring_TS[0])
ii = f_x(xmooring)
ji = f_y(ymooring)
xo = ds.interp(i=ii, j=ji, k=ki)
xo.Theta.plot.line(x='time')
sdata.Theta.where(sdata.dep == sdata.dep.max(), drop=True).plot.line(x='time')
sgridded.Theta.isel(depth=0).plot()

In [ ]:
# test if the above regridding is correct by plotting the data at one level as a time series
# YES.

# shallowest (max because negative)
zpl = np.max(sdata['dep'].data)
print(zpl)

# select data at one depth:
this_var_onedep = sdata[vbl].where(sdata.dep == zpl, drop=True)

# fixed data at the same depth
this_var_fixed_onedep = this_var_fix[0,:]


# # deepest
# zpl = np.min(sdata['dep'].data)
# this_var_onedep = sdata[vbl].where(sdata.dep == zpl, drop=True)
# this_var_reshaped_onedep = this_var_reshape[494,:]
# print(zpl)


tpl = np.linspace(0,np.max(sdata['time']),len(this_var_reshaped_onedep))


plt.plot(sdata['time'].where(sdata.dep == zpl, drop=True).data, this_var_onedep.data,'.')
plt.plot(tpl,this_var_fixed_onedep,'r-.')

plt.show()